# 使用LangChain构建本地RAG应用

该教程假设您已经熟悉以下概念:

- [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models)
- [Chaining runnables](https://python.langchain.com/v0.2/docs/how_to/sequence/)
- [Embeddings](https://python.langchain.com/v0.2/docs/concepts/#embedding-models)
- [Vector stores](https://python.langchain.com/v0.2/docs/concepts/#vector-stores)
- [Retrieval-augmented generation](https://python.langchain.com/v0.2/docs/tutorials/rag/)


很多流行的项目如 [llama.cpp](https://github.com/ggerganov/llama.cpp), [Ollama](https://github.com/ollama/ollama), 和 [llamafile](https://github.com/Mozilla-Ocho/llamafile) 显示了本地环境中运行大语言模型的重要性。

LangChain 与许多可以本地运行的 [开源 LLM 供应商](https://python.langchain.com/v0.2/docs/how_to/local_llms) 有集成，[Ollama](https://python.langchain.com/v0.2/docs/integrations/providers/ollama/) 便是其中之一。


## 环境设置

首先，我们需要进行环境设置。

Ollama 的 [GitHub仓库](https://github.com/ollama/ollama) 中提供了详细的说明, 简单总结如下:

- [下载](https://ollama.com/download) 并运行 Ollama 应用程序
- 从命令行, 参考 [Ollama 模型列表](https://ollama.com/library) 和 [文本嵌入模型列表](https://python.langchain.com/v0.2/docs/integrations/text_embedding/) 拉取模型。在该教程中，我们以 `llama3.1:8b` 和 `nomic-embed-text` 为例:
  - 命令行输入 `ollama pull llama3.1:8b`，拉取通用的开源大语言模型 `llama3.1:8b` 
  - 命令行输入 `ollama pull nomic-embed-text` 拉取 [文本嵌入模型](https://ollama.com/search?c=embedding) `nomic-embed-text`
- 当应用运行时，所有模型将自动在 `localhost:11434` 上启动
- 注意，你的模型选择需要考虑你的本地硬件能力，该教程的参考显存大小 `GPU Memory > 8GB`

接下来，安装本地嵌入、向量存储和模型推理所需的包。

In [1]:
# langchain_community
%pip install -qU langchain langchain_community

# Chroma
%pip install -qU langchain_chroma

# Ollama
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


You can also [see this page](/docs/integrations/text_embedding/) for a full list of available embeddings models

## 文档加载

现在让我们加载并分割一个示例文档。

我们将以 Lilian Weng 的关于 Agent 的 [博客](https://lilianweng.github.io/posts/2023-06-23-agent/) 为例。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

接着，初始化向量存储。 我们使用的文本嵌入模型是 [`nomic-embed-text`](https://ollama.com/library/nomic-embed-text) 。

In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

现在我们得到了一个本地的向量数据库! 来简单测试一下相似度检索:

In [4]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [5]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

接下来实例化大语言模型 `llama3.1:8b` 并测试模型推理是否正常：

In [6]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [7]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

**The scene is set: a packed arena, the crowd on their feet. In the blue corner, we have Stephen Colbert, aka "The O'Reilly Factor" himself. In the red corner, the challenger, John Oliver. The judges are announced as Tina Fey, Larry Wilmore, and Patton Oswalt. The crowd roars as the two opponents face off.**

**Stephen Colbert (aka "The Truth with a Twist"):**
Yo, I'm the king of satire, the one they all fear
My show's on late, but my jokes are clear
I skewer the politicians, with precision and might
They tremble at my wit, day and night

**John Oliver:**
Hold up, Stevie boy, you may have had your time
But I'm the new kid on the block, with a different prime
Time to wake up from that 90s coma, son
My show's got bite, and my facts are never done

**Stephen Colbert:**
Oh, so you think you're the one, with the "Last Week" crown
But your jokes are stale, like the ones I wore down
I'm the master of absurdity, the lord of the spin
You're just a British import, trying to fit in

**John Oliver

## 构建 Chain 表达形式

我们可以通过传入检索到的文档和简单的 prompt 来构建一个 `summarization chain` 。

它使用提供的输入键值格式化提示模板，并将格式化后的字符串传递给指定的模型：

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# 将传入的文档转换成字符串的形式
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these documents are:\n\n1. **Task Decomposition**: The process of breaking down complex tasks into smaller, manageable subgoals is crucial for efficient task handling.\n2. **Autonomous Agent System**: A system powered by Large Language Models (LLMs) that can perform planning, reflection, and refinement to improve the quality of final results.\n3. **Challenges in Planning and Decomposition**:\n\t* Long-term planning and task decomposition are challenging for LLMs.\n\t* Adjusting plans when faced with unexpected errors is difficult for LLMs.\n\t* Humans learn from trial and error, making them more robust than LLMs in certain situations.\n\nOverall, the documents highlight the importance of task decomposition and planning in autonomous agent systems powered by LLMs, as well as the challenges that still need to be addressed.'

## 简单QA


In [9]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'Task decomposition can be done through (1) simple prompting using LLM, (2) task-specific instructions, or (3) human inputs. This approach helps break down large tasks into smaller, manageable subgoals for efficient handling of complex tasks. It enables agents to plan ahead and improve the quality of final results through reflection and refinement.'

## 带有检索的QA

最后，我们带有语义检索功能的 QA 应用（本地 RAG 应用），可以根据用户问题自动从向量数据库中检索语义上最相近的文档片段：

In [10]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [11]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'Task decomposition can be done through (1) simple prompting in Large Language Models (LLM), (2) using task-specific instructions, or (3) with human inputs. This process involves breaking down large tasks into smaller, manageable subgoals for efficient handling of complex tasks.'

## 总结

恭喜，至此，你已经完整的实现了一个基于 Langchain 框架和本地模型构建的 RAG 应用。你可以在教程的基础上替换本地模型来尝试不同模型的效果和能力，或进一步进行扩展，丰富应用的能力和表现力，或者添加更多实用有趣的功能。